Subset of https://gist.github.com/genkuroki/8c7e9e484a5877d9747a4819bbab645d

In [1]:
function conseclist_forloop(x)
    b, e = firstindex(x), lastindex(x)
    s = b - 1
    S = [s]
    while s < e
        t = s + 1
        @inbounds while t < e
            x[t+1] != x[t] + 1 && break 
            t += 1
        end
        push!(S, t)
        s = t
    end
    [@view(x[S[i]+1:S[i+1]]) for i in 1:length(S)-1]
end

conseclist_forloop (generic function with 1 method)

In [2]:
function conseclist_bitvector(x)
    b, e = firstindex(x), lastindex(x)
    @views idxs = x[b+1:e] .- x[b:e-1] .!= 1
    S = [b-1; (b:e-1)[idxs]; e]
    [@view(x[S[i]+1:S[i+1]]) for i in 1:length(S)-1]
end

conseclist_bitvector (generic function with 1 method)

In [3]:
module LazyConsecLists

struct ConsecList{T<:AbstractVector{<:Integer}} <:
        AbstractVector{SubArray{eltype(T), 1, T, Tuple{UnitRange{Int64}}, true}}
    a::T
    S::Vector{Int}
end
function ConsecList(x)
    b, e = firstindex(x), lastindex(x)
    idx = @views (x[b+1:e] .- x[b:e-1]) .!= 1
    S = [b-1; (b:e-1)[idx]; e]
    ConsecList{typeof(x)}(x, S)
end

Base.length(x::ConsecList) = length(x.S) - 1
Base.size(x::ConsecList) = (length(x),)
function Base.eltype(x::ConsecList{T}) where T<:AbstractVector{<:Integer}
    SubArray{eltype(T), 1, T, Tuple{UnitRange{Int64}}, true}
end

Base.getindex(x::ConsecList, i::Integer) = @view(x.a[x.S[i]+1:x.S[i+1]])
Base.getindex(x::ConsecList, r::AbstractRange) = [x[i] for i in r]
Base.getindex(x::ConsecList, ::Colon) = collect(x)

end

Main.LazyConsecLists

In [4]:
using BenchmarkTools
A = unique(sort(rand(1:10^4, 10^4)))

6251-element Vector{Int64}:
     1
     2
     3
     6
     7
     8
     9
    10
    11
    12
    13
    14
    15
     ⋮
  9983
  9985
  9986
  9987
  9990
  9991
  9992
  9993
  9995
  9996
  9997
 10000

In [5]:
@show conseclist_forloop(A) == conseclist_bitvector(A) == LazyConsecLists.ConsecList(A)
@show length(conseclist_forloop(A)) == length(conseclist_bitvector(A)) == length(LazyConsecLists.ConsecList(A))
@btime conseclist_forloop($A)
@btime conseclist_bitvector($A)
@btime collect(LazyConsecLists.ConsecList($A))

conseclist_forloop(A) == conseclist_bitvector(A) == LazyConsecLists.ConsecList(A) = true
length(conseclist_forloop(A)) == length(conseclist_bitvector(A)) == length(LazyConsecLists.ConsecList(A)) = true
  41.000 μs (14 allocations: 157.23 KiB)
  16.900 μs (9 allocations: 135.08 KiB)
  17.300 μs (9 allocations: 135.08 KiB)


2371-element Vector{SubArray{Int64, 1, Vector{Int64}, Tuple{UnitRange{Int64}}, true}}:
 [1, 2, 3]
 [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
 [20]
 [22, 23, 24, 25, 26, 27, 28]
 [30, 31]
 [33, 34, 35, 36, 37]
 [40, 41, 42]
 [45, 46, 47, 48, 49]
 [54]
 [58, 59, 60, 61]
 [63]
 [65, 66, 67, 68]
 [71, 72]
 ⋮
 [9943, 9944, 9945, 9946, 9947, 9948, 9949, 9950]
 [9952, 9953, 9954]
 [9956, 9957, 9958, 9959, 9960, 9961, 9962]
 [9964, 9965]
 [9967, 9968, 9969]
 [9972]
 [9974, 9975, 9976, 9977, 9978, 9979]
 [9981, 9982, 9983]
 [9985, 9986, 9987]
 [9990, 9991, 9992, 9993]
 [9995, 9996, 9997]
 [10000]

In [6]:
@btime sum(length, conseclist_forloop($A))
@btime sum(length, conseclist_bitvector($A))
@btime sum(length, LazyConsecLists.ConsecList($A))

  42.200 μs (14 allocations: 157.23 KiB)
  17.700 μs (9 allocations: 135.08 KiB)
  10.800 μs (7 allocations: 42.38 KiB)


6251